In [60]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


import warnings
warnings.filterwarnings("ignore")

############################################## exploration et traitements des données #####################################

# Remplacez 'votre_fichier.xlsx' par le nom de votre fichier Excel
excel_file = "Analyse TB2023.xlsx"

# Remplacez 'Feuille1' et 'Feuille3' par les noms des feuilles que vous souhaitez consolider
sheets_to_consolidate = ['AVRIL 23', 'MAI 23', 'JUIN 23', 'JUILLET 23'] 

# Charger les feuilles spécifiées en un dictionnaire de DataFrames
dfs = pd.read_excel(excel_file, sheet_name=sheets_to_consolidate)

# Concaténer les DataFrames en utilisant les noms d'entête de colonnes
consolidated_sheet = pd.concat(dfs, ignore_index=True, sort=False)

# Convertir la colonne "Date " en format de date avec gestion des erreurs
consolidated_sheet["Date "] = pd.to_datetime(consolidated_sheet["Date "], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Filtrer les lignes où la colonne "Date " est au format de date valide
consolidated_sheet = consolidated_sheet[consolidated_sheet["Date "].notnull()]

# Sélectionner uniquement les colonnes "Date ", "CA" et "ACHATS"
columns_to_keep = ["Date ", "CA", "ACHATS"]
consolidated_sheet = consolidated_sheet[columns_to_keep]

# Enregistrer la feuille consolidée avec les colonnes spécifiques dans un nouveau fichier Excel
consolidated_excel_file = 'feuille_consolid.xlsx'
consolidated_sheet.to_excel(consolidated_excel_file, index=False)


################################################### 2 #############################################################################

# Ouvrir le fichier Excel "Analyse TB2023.xlsx" et charger la feuille "Détail Dépenses"
excel_file = "Analyse TB2023.xlsx"
sheet_name =  "Détail Dépenses"

df = pd.read_excel(excel_file, sheet_name)

# Transformer la colonne "Date " en format de date avec gestion des erreurs
df["Date "] = pd.to_datetime(df["Date "], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Supprimer les lignes où la colonne "Date " n'est pas au format "date"
df = df.dropna(subset=["Date "])

# Supprimer les colonnes "Unnamed: 25" et "JOURS"
columns_to_drop = ["Unnamed: 25", "JOURS","TOTAL DEPENSES "]
df = df.drop(columns=columns_to_drop)

# Enregistrer les données dans un nouveau fichier Excel "Analyse.xlsx"
new_excel_file = "Analyse.xlsx"

df.to_excel(new_excel_file, index=False)

################################################ 3 ###############################################################################
# Fusionner les DataFrames df et consolidated_sheet sur la colonne "Date "
merged_df = df.merge(consolidated_sheet[["Date ", "CA", "ACHATS"]], on="Date ", how="left")



# Remplacer les valeurs NaN par 0 dans tout le DataFrame
merged_df.fillna(0, inplace=True)


# Enregistrer les données dans un nouveau fichier Excel "Analyse.xlsx"
new_excel_file = "Analyse_Globale.xlsx"

merged_df.to_excel(new_excel_file, index=False)

print(f"Données de la feuille '{sheet_name}' traitées et enregistrées dans '{new_excel_file}'.")


###################################################  4  #######################################################################

# Charger les données de la feuille "RH" depuis le fichier Excel
excel_file = "Analyse TB2023.xlsx"
sheet_name = "RH"
df = pd.read_excel(excel_file, sheet_name)

# Ouvrir un nouveau fichier Excel pour sauvegarder le contenu traité
new_excel_file = "ChargePersonnel.xlsx"
with pd.ExcelWriter(new_excel_file, engine="xlsxwriter") as writer:
    # Enregistrer le DataFrame d'origine dans le nouveau fichier Excel
    df.to_excel(writer, sheet_name="RH", index=False)

# Lire le nouveau fichier Excel pour le traitement ultérieur
df_new = pd.read_excel(new_excel_file, sheet_name="RH")

# Sélectionner uniquement la colonne "Date" et la colonne "ChargePersonnel"
df_filtered = df_new[['Date ', "ChargePersonnel"]]

# Supprimer les lignes vides ou égales à 0
df_filtered = df_filtered.dropna(subset=['Date ', "ChargePersonnel"])
df_filtered = df_filtered[(df_filtered != 0).all(axis=1)]

# Transformer la colonne "Date" au format de date
df_filtered['Date '] = pd.to_datetime(df_filtered['Date '], format='%d/%m/%Y')  # Adapter le format au format réel dans votre fichier

# Enregistrer le DataFrame filtré dans un autre fichier Excel
new_filtered_excel_file = "ChargePersonnel.xlsx"
df_filtered.to_excel(new_filtered_excel_file, index=False)

print(f"Données filtrées enregistrées dans '{new_filtered_excel_file}'.")
#df_filtered

merged = merged_df.merge(df_filtered[["Date ", "ChargePersonnel"]], on="Date ", how="left")

merged.fillna(0, inplace=True)

merged


# Enregistrer le DataFrame filtré dans un autre fichier Excel
new_filtered_excel_file = "DétailsDépenses.xlsx"
merged.to_excel(new_filtered_excel_file, index=False)

print(f"Données consolidées et enregistrées dans '{new_filtered_excel_file}'.")
#df_filtered

#merged


########################################### calcul financier #################################################################


merged_copy = merged.copy()
df0 = merged_copy

########Calculer le total par colonne :#########

# Somme de chaque colonne
total_par_colonne = df0.sum()

# Somme de chaque ligne
total_par_ligne = df0.sum(axis=1)


######Coûts des produits vendus##########

# Liste des colonnes à inclure dans le calcul
colonnes_a_inclure = ["SMOKE", "EAT", "GAZ", "DRINK", "MIAMI 228 ", "PICASSO", "GLACONS"]

# Ajouter une colonne "Coûts des produits vendus"
df0["Coûts des produits vendus"] = df0[colonnes_a_inclure].sum(axis=1)



##################################################

########## Marge brute#######

# Créer la nouvelle colonne "Marge brute"
df0["Marge brute"] = df0["CA"] - df0["Coûts des produits vendus"]

########## Charge operationnel ##########

# Liste des colonnes à inclure dans le calcul
colonnesinclure = ['MONNAIE', 'CACHETS', 'MARKETING', 'CASH POWER',
       'CREDIT TEL', 'INTERNET / TV', 'LOYERS', 'ADMINISTRATIF',
       'CONSOMMABLES', 'ENTRETIEN ', 'TRANSPORT', 'AUTRE',  'ACHATS', 'ChargePersonnel']

# Ajouter une colonne "OPEX"
df0["OPEX"] = df0[colonnesinclure].sum(axis=1)

########## Resultat d'exploitation ##########

df0["Resultat d'exploitation"] = df0["Marge brute"] - df0["OPEX"]

########## Resultat avant Impôts ##########

ChargesInterets = 0
df0["Resultat avant Impôts"] = df0["Resultat d'exploitation"] - ChargesInterets

########## Resultat net comptable ##########

Taxes = 0
df0["Resultat net comptable"] = df0["Resultat avant Impôts"] - Taxes

 ########## Tresorerie net d'exploitation ##########


df0["Tresorerie net d'exploitation"] = df0["Resultat net comptable"]

########## Travaux et equipements ##########

# Liste des colonnes à inclure dans le calcul
colonnescal = ['EQUIPEMENTS','TRAVAUX']

# Ajouter une colonne "Coûts des produits vendus"
df0["Travaux et equipements"] = df0[colonnescal].sum(axis=1)

########## Tresorerie net d'investissement ##########

df0["Tresorerie net d'investissement"] = df0["Travaux et equipements"]

########## Resultat net ##########

df0["Resultat net"] = df0["Tresorerie net d'exploitation"] + df0["Tresorerie net d'investissement"]

########## Working Capital ##########

df0["Working Capital"] = df0["ACHATS"]

########## Trésorerie Fin de Mois ##########

df0["Trésorerie Fin de Mois"] = df0["Working Capital"]

########## Taux marge brute ##########

df0["Taux marge brute"] = df0["Marge brute"]/df0["CA"]


# Liste des colonnes à inclure dans le calcul
col = ['MONNAIE', 
       'CREDIT TEL', 'INTERNET / TV', 'LOYERS', 'ADMINISTRATIF',
       'CONSOMMABLES', 'ENTRETIEN ', 'TRANSPORT', 'AUTRE']

# Ajouter une colonne "Autres"
df0["Autres"] = df0[col].sum(axis=1)


# Liste des colonnes à inclure dans le calcul
col1 = [ 'MARKETING', 'ADMINISTRATIF']

# Ajouter une colonne "MARKETINGADMINISTRATIF"
df0["MARKETINGADMINISTRATIF"] = df0[col1].sum(axis=1)


# Liste des colonnes à afficher
#columns_to_display = ['Date ', "Coûts des produits vendus", "Marge brute", "OPEX", "Resultat d'exploitation","Resultat avant Impôts",
#                     "Resultat net comptable","Tresorerie net d'exploitation","Travaux et equipements","Tresorerie net d'investissement",
#                     "Resultat net","Working Capital","Trésorerie Fin de Mois","Taux marge brute"]

# Créer un nouveau DataFrame avec uniquement les colonnes à afficher
#df_subset = df0[columns_to_display]

# Afficher le DataFrame résultat
#df_subset

############################################ TCD #############################################################

# Convertir la colonne 'Date' en type datetime
df0['Date '] = pd.to_datetime(df0['Date '])

# Extraire le mois et l'année à partir de la colonne 'Date'
df0['Mois'] = df0['Date '].dt.to_period('M')

# Liste des colonnes pour le TCD
columns_for_tcd = ['Mois', "CA",'Coûts des produits vendus', 'Marge brute','CACHETS', 'CASH POWER',
                   "MARKETINGADMINISTRATIF",'ChargePersonnel',"Autres",'OPEX', 'Resultat d\'exploitation',
                   'Resultat avant Impôts', 'Resultat net comptable', 'Tresorerie net d\'exploitation',
                   'Travaux et equipements', 'Tresorerie net d\'investissement', 'Resultat net',
                   'Working Capital', 'Trésorerie Fin de Mois', 'Taux marge brute']

# Créer le TCD en groupant par mois
tcd = df0[columns_for_tcd].groupby('Mois').sum()

# Afficher le tableau croisé dynamique
tcd



Données de la feuille 'Détail Dépenses' traitées et enregistrées dans 'Analyse_Globale.xlsx'.
Données filtrées enregistrées dans 'ChargePersonnel.xlsx'.
Données consolidées et enregistrées dans 'DétailsDépenses.xlsx'.


,CA,Coûts des produits vendus,Marge brute,CACHETS,CASH POWER,MARKETINGADMINISTRATIF,ChargePersonnel,Autres,OPEX,Resultat d'exploitation,Resultat avant Impôts,Resultat net comptable,Tresorerie net d'exploitation,Travaux et equipements,Tresorerie net d'investissement,Resultat net,Working Capital,Trésorerie Fin de Mois,Taux marge brute
Mois,,,,,,,,,,,,,,,,,,,
2023-04,3527300.0,2973450.0,553850.0,20000.0,0.0,145100.0,549500.0,237100.00,4.191150e+06,-3.637300e+06,-3.637300e+06,-3.637300e+06,-3.637300e+06,1.140000e+05,1.140000e+05,-3523300.00,3.344550e+06,3.344550e+06,0.549997
2023-05,12225050.0,6179050.0,6046000.0,335000.0,470000.0,227700.0,1399000.0,889822.26,1.430902e+07,-8.263022e+06,-8.263022e+06,-8.263022e+06,-8.263022e+06,3.087027e+06,3.087027e+06,-5175994.52,1.109005e+07,1.109005e+07,11.310243
2023-06,20471000.0,8370550.0,12100450.0,480000.0,550000.0,48300.0,1736000.0,809300.00,1.575715e+07,-3.656700e+06,-3.656700e+06,-3.656700e+06,-3.656700e+06,1.972000e+06,1.972000e+06,-1684700.00,1.218185e+07,1.218185e+07,14.028692
2023-07,28821750.0,15470625.0,13351125.0,630000.0,500000.0,177600.0,1809500.0,805325.00,2.251008e+07,-9.158950e+06,-9.158950e+06,-9.158950e+06,-9.158950e+06,1.159300e+06,1.159300e+06,-7999650.00,1.871525e+07,1.871525e+07,12.624399


In [61]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px


a = tcd.copy()
# Charger vos données
df = a

# Convertir les index en chaînes de caractères au format YYYY-MM
df.index = df.index.strftime('%Y-%m')

# Calcul du taux de marge brute
df['Taux marge brute'] = df['Taux marge brute'] * 100

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Création des graphiques
fig1 = px.bar(df, x=df.index, y='CA', title='CA en bar')
fig2 = px.line(df, x=df.index, y='Taux marge brute', title='Taux de marge brute en courbe')

fig2.update_yaxes(title_text='% Taux de marge brute', secondary_y=True)

fig3 = px.line(df, x=df.index, y=['CACHETS', 'CASH POWER', 'MARKETINGADMINISTRATIF', 'ChargePersonnel', 'Autres'],
               title='Evolution des charges opérationnelles')

fig4 = px.pie(df, names=df.index, values='CA', title='Répartition du CA sur les mois')

fig5 = px.line(df, x=df.index, y='Coûts des produits vendus', title="Évolution des coûts de produit")

fig6 = px.line(df, x=df.index, y=['Tresorerie net d\'exploitation', 'Tresorerie net d\'investissement'],
               title="Flux de trésorerie")

# Mise en page de l'application Dash
app.layout = html.Div([
    html.Div([dcc.Graph(figure=fig1)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig2)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig3)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig4)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig5)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig6)], className='graph-container')
])

# Exécution de l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


In [52]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px

# Charger vos données
# df = tcd

# Calcul du taux de marge brute
df['Taux marge brute'] = df['Taux marge brute'] * 100

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Création des graphiques
fig1 = px.bar(df, x=df.index, y='CA', title='CA en bar')
fig2 = px.line(df, x=df.index, y='Taux marge brute', title='Taux de marge brute en courbe')

fig2.update_yaxes(title_text='% Taux de marge brute', secondary_y=True)

fig3 = px.line(df, x=df.index, y=['CACHETS', 'CASH POWER', 'MARKETINGADMINISTRATIF', 'ChargePersonnel', 'Autres'],
               title='Evolution des charges opérationnelles')

fig4 = px.pie(df, names=df.index, values='CA', title='Répartition du CA sur les mois')

fig5 = px.line(df, x=df.index, y='Coûts des produits vendus', title="Évolution des coûts de produit")

fig6 = px.line(df, x=df.index, y=['Tresorerie net d\'exploitation', 'Tresorerie net d\'investissement'],
               title="Flux de trésorerie")

# Mise en page de l'application Dash
app.layout = html.Div([
    html.Div([dcc.Graph(figure=fig1)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig2)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig3)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig4)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig5)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig6)], className='graph-container')
])

# Exécution de l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
~\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of <Flask '__main__'>>
   1951         except Exception as e:
   1952             rv = self.handle_user_exception(e)

~\anaconda3\lib\site-packages\flask\app.py in dispatch_request(self=<Flask '__main__'>)
   1934             return self.make_default_options_response()
   1935         # otherwise dispatch to the handler for that endpoint
-> 1936         return self.view_functions[rule.endpoint](**req.view_args)
        self.view_functions = {'static': <bound method _PackageBoundObject.send_static_file of <Flask '__main__'>>

In [51]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px

# Charger vos données
df = tcd

# Calcul du taux de marge brute
df['Taux marge brute'] = df['Taux marge brute'] * 100

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Création des graphiques
fig1 = px.bar(df, x=df.index, y=['CA'], title='CA en bar et Taux de marge brute en courbe')
fig1.add_trace(px.line(df, x=df.index, y=['Taux marge brute'], yaxis='y2').data[0])
fig1.update_layout(yaxis2=dict(overlaying='y', side='right'))

fig2 = px.line(df, x=df.index, y=['CACHETS', 'CASH POWER', 'MARKETINGADMINISTRATIF', 'ChargePersonnel', 'Autres'],
               title='Evolution des charges opérationnelles')

fig3 = px.pie(df, names=df.index, values='CA', title='Répartition du CA sur les mois')

fig4 = px.line(df, x=df.index, y=['Coûts des produits vendus'], title="Évolution des coûts de produit")

fig5 = px.line(df, x=df.index, y=['Tresorerie net d\'exploitation', 'Tresorerie net d\'investissement'],
               title="Flux de trésorerie")

# Mise en page de l'application Dash
app.layout = html.Div([
    html.Div([dcc.Graph(figure=fig1)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig2)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig3)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig4)], className='graph-container'),
    html.Div([dcc.Graph(figure=fig5)], className='graph-container')
])

# Exécution de l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


TypeError: line() got an unexpected keyword argument 'yaxis'

In [49]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


# Créer une application Dash
app = dash.Dash(__name__)

# Charger les données
data = tcd
df = pd.DataFrame(data)
df.set_index("Mois", inplace=True)

# Définir les fonctions pour générer les graphiques
def generate_revenue_gross_margin():
    fig = px.bar(df, x=df.index, y="CA", title="Chiffre d'affaires et Marge brute")
    fig.add_trace(go.Scatter(x=df.index, y=df['Marge brute'], mode='lines', name='Marge brute'))

    fig.update_layout(xaxis_type='category')

    return fig





def generate_operational_expenses_treemap():
    expenses_columns = ["CACHETS", "CASH POWER", "MARKETINGADMINISTRATIF", "ChargePersonnel", "Autres"]
    expenses_df = df[expenses_columns].reset_index()
    expenses_df = pd.melt(expenses_df, id_vars=["Mois"], value_vars=expenses_columns,
                          var_name="Category", value_name="Amount")
    fig = px.treemap(expenses_df, path=['Category', 'Mois'], values='Amount', title='Évolution des charges opérationnelles')
    return fig

def generate_monthly_revenue_distribution():
    fig = px.pie(df, names=df.index, values='CA', title='Répartition du chiffre d\'affaires sur les mois')
    return fig

def generate_product_costs_evolution():
    fig = px.line(df, x=df.index, y='Coûts des produits vendus', title='Évolution des coûts de produit')
    return fig

def generate_cash_flow():
    cash_flow_columns = ["Tresorerie net d'exploitation", "Tresorerie net d'investissement"]
    cash_flow_df = df[cash_flow_columns].reset_index()
    fig = px.area(cash_flow_df, x='Mois', y=cash_flow_columns, title='Flux de trésorerie')
    return fig

# Mise en page de l'application Dash
app.layout = html.Div([
    dcc.Graph(id='revenue-gross-margin', figure=generate_revenue_gross_margin()),
    dcc.Graph(id='operational-expenses-treemap', figure=generate_operational_expenses_treemap()),
    dcc.Graph(id='monthly-revenue-distribution', figure=generate_monthly_revenue_distribution()),
    dcc.Graph(id='product-costs-evolution', figure=generate_product_costs_evolution()),
    dcc.Graph(id='cash-flow', figure=generate_cash_flow())
])

# Exécution de l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


KeyError: "None of ['Mois'] are in the columns"

In [26]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Créer le DataFrame à partir des données fournies
data = {
    'Mois': ['2023-04', '2023-05', '2023-06', '2023-07'],
    'Coûts des produits vendus': [2.973450e+06, 6.179050e+06, 8.370550e+06, 1.547062e+07],
    'Marge brute': [5.538500e+05, 6.046000e+06, 1.210045e+07, 1.335112e+07],
    'OPEX': [4.191150e+06, 1.430902e+07, 1.575715e+07, 2.251008e+07],
    'Resultat d\'exploitation': [-3.637300e+06, -8.263022e+06, -3.656700e+06, -9.158950e+06],
    'Resultat avant Impôts': [-3.637300e+06, -8.263022e+06, -3.656700e+06, -9.158950e+06],
    'Resultat net comptable': [-3.637300e+06, -8.263022e+06, -3.656700e+06, -9.158950e+06],
    'Tresorerie net d\'exploitation': [-3.637300e+06, -8.263022e+06, -3.656700e+06, -9.158950e+06],
    'Travaux et equipements': [1.140000e+05, 3.087027e+06, 1.972000e+06, 1.159300e+06],
    'Tresorerie net d\'investissement': [1.140000e+05, 3.087027e+06, 1.972000e+06, 1.159300e+06],
    'Resultat net': [-3.523300e+06, -5.175995e+06, -1.684700e+06, -7.999650e+06],
    'Working Capital': [3.344550e+06, 1.109005e+07, 1.218185e+07, 1.871525e+07],
    'Trésorerie Fin de Mois': [3.344550e+06, 1.109005e+07, 1.218185e+07, 1.871525e+07],
    'Taux marge brute': [5.499970e-01, 1.131024e+01, 1.402869e+01, 1.262440e+01]
}

df = pd.DataFrame(data)

# Créer une application Dash
app = dash.Dash(__name__)

# Fonction pour générer le graphique CA et Taux de marge brute
def generate_ca_margin_graph():
    fig = px.line(df, x='Mois', y=['Coûts des produits vendus', 'Marge brute'],
                  labels={'value': 'Montant', 'variable': 'Catégorie'},
                  title='Coûts des produits vendus et Marge brute')
    return fig

# Mise en page de l'application Dash
app.layout = html.Div([
    dcc.Graph(id='ca-margin-graph', figure=generate_ca_margin_graph())
])

# Exécution de l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=5500)
